In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate
import casadi
from autoware_vehicle_adaptor.param import parameters

In [ ]:
max_velocity = 11.1 # 11m/s=40km/h # max velocity limit [m/s]
max_lateral_accel = 0.8 # max lateral acceleration limit [m/ss] # https://github.com/autowarefoundation/autoware.universe/blob/98c24b824c68605c28513de47a36aaf5a4a89817/planning/motion_velocity_smoother/config/default_motion_velocity_smoother.param.yaml#L15
min_acc = -1.0
max_acc = 1.0
min_jerk = -1.0
max_jerk = 1.0
max_steering_angle_rate = 40.0 # maximum steering angle rate [degree/s] # https://github.com/autowarefoundation/autoware.universe/blob/98c24b824c68605c28513de47a36aaf5a4a89817/planning/motion_velocity_smoother/config/default_motion_velocity_smoother.param.yaml#L22C5-L22C86
wheel_base = parameters.wheel_base

In [ ]:
# path parameter
dpos = 0.01
dsteer_dpos_max = 0.04

In [ ]:
def get_left_turn_path():
    x = 0.0
    y = 0.0
    yaw = 0.0
    steer = 0.0
    temp_path = []
    temp_path.append([x,y,yaw,steer])
    while True:
        x += dpos * np.cos(yaw)
        y += dpos * np.sin(yaw)
        yaw += dpos * np.tan(steer) / wheel_base
        steer += dsteer_dpos_max * dpos
        temp_path.append([x,y,yaw,steer])
        if yaw >= np.pi / 4:
            break
    if np.pi / 4 - temp_path[-2][2] < temp_path[-1][2] - np.pi / 4:
        temp_path.pop()
    states_prev = temp_path[-1]
    x_prev = states_prev[0]
    y_prev = states_prev[1]
    intercept = (x_prev + 0.5 * dpos * np.cos(np.pi / 4)) + (y_prev + 0.5 * dpos * np.sin(np.pi / 4))
    temp_path_head = temp_path.copy()    
    for i in range(1, len(temp_path) + 1):
        states_inverse = temp_path_head[- i]
        x_inverse = states_inverse[0]
        y_inverse = states_inverse[1]
        yaw_inverse = states_inverse[2]
        steer_inverse = states_inverse[3]
        x = -y_inverse + intercept
        y = -x_inverse + intercept
        yaw = np.pi / 2 - yaw_inverse
        steer = steer_inverse
        temp_path.append([x,y,yaw,steer])
    return np.array(temp_path)
def get_right_turn_path():
    temp_path = []
    left_turn_path = get_left_turn_path()
    for states in left_turn_path:
        x_left = states[0]
        y_left = states[1]
        yaw_left = states[2]
        steer_left = states[3]
        temp_path.append([-x_left, y_left, np.pi - yaw_left, - steer_left])
    return np.array(temp_path)
        

In [ ]:
left_turn_path = get_left_turn_path()
plt.plot(left_turn_path[:,0],left_turn_path[:,1])
plt.axis('equal')
plt.show()
plt.plot(dpos * np.arange(left_turn_path.shape[0]),left_turn_path[:,2])
plt.show()
plt.plot(dpos * np.arange(left_turn_path.shape[0]),left_turn_path[:,3])
plt.show()

In [ ]:
right_turn_path = get_right_turn_path()
plt.plot(right_turn_path[:,0],right_turn_path[:,1])
plt.axis('equal')
plt.show()
plt.plot(dpos * np.arange(right_turn_path.shape[0]),right_turn_path[:,2])
plt.show()
plt.plot(dpos * np.arange(right_turn_path.shape[0]),right_turn_path[:,3])
plt.show()

In [ ]:
def get_total_path(x_length,y_length):
    left_turn_path = get_left_turn_path()
    right_turn_path = get_right_turn_path()
    x_num = int(x_length // dpos)
    y_num = int(y_length // dpos)
    x = 0.0
    y = 0.0
    yaw = 0.0
    steer = 0.0
    total_path = []
    total_path.append(np.array([x,y,yaw,steer]))
    for i in range(2 * x_num + 2*int((left_turn_path[-1,0] - left_turn_path[0,0])//dpos)):
        x += dpos
        total_path.append(np.array([x,y,yaw,steer]))
    first_position = total_path[-1]
    for i in range(1,len(left_turn_path)):
        states = left_turn_path[i]
        x = states[0] + first_position[0]
        y = states[1] + first_position[1]
        yaw = states[2]
        steer = states[3]
        total_path.append(np.array([x,y,yaw,steer]))
    second_position = total_path[-1]
    for i in range(y_num):
        x = second_position[0]
        y = second_position[1] + (i+1)*dpos
        yaw = second_position[2]
        steer = second_position[3]
        total_path.append(np.array([x,y,yaw,steer]))
    third_position = total_path[-1]
    for i in range(1,len(left_turn_path)):
        states = left_turn_path[i]
        x = - states[1] + third_position[0]
        y = states[0] + third_position[1]
        yaw = states[2] + np.pi / 2
        steer = states[3]
        total_path.append(np.array([x,y,yaw,steer]))       
    forth_position = total_path[-1]
    for i in range(x_num):
        x = forth_position[0] - (i+1)*dpos
        y = forth_position[1]
        yaw = forth_position[2]
        steer = forth_position[3]
        total_path.append(np.array([x,y,yaw,steer]))
    fifth_position = total_path[-1]
    stopping_target = len(total_path) - 1
    for i in range(1,len(left_turn_path)):
        states = left_turn_path[i]
        x = - states[0] + fifth_position[0]
        y = - states[1] + fifth_position[1]
        yaw = states[2] + np.pi
        steer = states[3]
        total_path.append(np.array([x,y,yaw,steer]))
    sixth_position = total_path[-1]
    for i in range(y_num):
        x = sixth_position[0]
        y = sixth_position[1] - (i+1)*dpos
        yaw = sixth_position[2]
        steer = sixth_position[3]
        total_path.append(np.array([x,y,yaw,steer]))
    seventh_position = total_path[-1]
    for i in range(1,len(right_turn_path)):
        states = right_turn_path[i]
        x = - states[1] + seventh_position[0]
        y = states[0] + seventh_position[1]
        yaw = states[2] + np.pi / 2
        steer = states[3]
        total_path.append(np.array([x,y,yaw,steer]))
    eighth_position = total_path[-1]
    for i in range(x_num):
        x = eighth_position[0] - (i+1)*dpos
        y = eighth_position[1]
        yaw = eighth_position[2]
        steer = eighth_position[3]
        total_path.append(np.array([x,y,yaw,steer]))
    ninth_position = total_path[-1]
    for i in range(1,len(right_turn_path)):
        states = right_turn_path[i]
        x = states[0] + ninth_position[0]
        y = states[1] + ninth_position[1]
        yaw = states[2]
        steer = states[3]
        total_path.append(np.array([x,y,yaw,steer]))
    tenth_position = total_path[-1]
    for i in range(y_num):
        x = tenth_position[0]
        y = tenth_position[1] + (i+1)*dpos
        yaw = tenth_position[2]
        steer = tenth_position[3]
        total_path.append(np.array([x,y,yaw,steer]))
    eleventh_position = total_path[-1]
    for i in range(1,len(right_turn_path)):
        states = right_turn_path[i]
        x = states[1] + eleventh_position[0]
        y = - states[0] + eleventh_position[1]
        yaw = states[2] - np.pi / 2
        steer = states[3]
        total_path.append(np.array([x,y,yaw,steer]))
    twelfth_position = total_path[-1]
    for i in range(2 * x_num + 2 * int((left_turn_path[-1,0] - left_turn_path[0,0])//dpos)):
        x = twelfth_position[0] + (i+1)*dpos
        y = twelfth_position[1]
        yaw = twelfth_position[2]
        steer = twelfth_position[3]
        total_path.append(np.array([x,y,yaw,steer]))   
    return np.array(total_path), stopping_target

In [ ]:
total_path, stopping_target = get_total_path(20.0,20.0)
plt.plot(total_path[:,0],total_path[:,1])
plt.axis('equal')
plt.show()
plt.plot(dpos * np.arange(total_path.shape[0]),total_path[:,2])
plt.show()
plt.plot(dpos * np.arange(total_path.shape[0]),total_path[:,3])
plt.show()

# Initial velocity limit

In [ ]:
velocity_limit = np.array([max_velocity]*len(total_path))
plt.plot(np.arange(total_path.shape[0]) * dpos, velocity_limit,label="external velocity limit")
plt.xlabel("travel distance",fontsize=15)
plt.ylabel("longitudinal velocity",fontsize=15)
plt.ylim([-1,max_velocity+1])
plt.legend()
plt.show()

In [ ]:
applied_velocity_limit = []
for i in range(len(total_path)):
    if i == stopping_target:
        applied_velocity_limit.append(1.0)
    else:
        applied_velocity_limit.append(np.inf)
applied_velocity_limit = np.array(applied_velocity_limit)
plt.plot(total_path[:,0],velocity_limit, label="current velocity limit")
plt.plot(total_path[:,0],applied_velocity_limit, label="stopping velocity limit")
plt.legend()
plt.xlabel("position x",fontsize=15)
plt.ylabel("longitudinal velocity",fontsize=15)
plt.ylim([-1,max_velocity+1])
velocity_limit = np.stack([velocity_limit,applied_velocity_limit]).min(axis=0)

# Applying lateral acceleration limit
$$
    max\_lateral\_accel \ge r \left(\frac{dyaw}{dt}\right)^2 = \frac{v^2}{r}
$$
$$
    \sqrt{r\times max\_lateral\_accel} \ge v
$$

In [ ]:
r_trajectory = wheel_base/np.abs(np.tan(total_path[:,3])+1e-16)
applied_velocity_limit = np.sqrt(max_lateral_accel*r_trajectory )

plt.plot(np.arange(total_path.shape[0]) * dpos,velocity_limit, label="current velocity limit")
plt.plot(np.arange(total_path.shape[0]) * dpos,applied_velocity_limit, label="lateral acceleration limit")

plt.legend()
plt.xlabel("travel distance",fontsize=15)
plt.ylabel("longitudinal velocity",fontsize=15)
plt.ylim([-1,max_velocity+1])
plt.show()

velocity_limit = np.stack([velocity_limit,applied_velocity_limit]).min(axis=0)

# Applying steering rate limit
$$
    steer\_rate\_limit \ge \left|\frac{dsteer}{dt}\right| = \left|\frac{dpos}{dt} \frac{dsteer}{dpos}\right| = v\left|\frac{dsteer}{dpos}\right|
$$
$$
    \frac{steer\_rate\_limit}{\left|\frac{dsteer}{dpos}\right|} \ge v
$$

In [ ]:
dsteer_dpos = (total_path[1:,3]-total_path[:-1,3])/dpos
dsteer_dpos = np.hstack([dsteer_dpos,dsteer_dpos[-1]])
applied_velocity_limit = (max_steering_angle_rate*np.pi/180.0) / np.abs(1e-16+dsteer_dpos)
print(applied_velocity_limit.shape)
plt.plot(np.arange(total_path.shape[0]) * dpos,velocity_limit, label="current velocity limit")
plt.plot(np.arange(total_path.shape[0]) * dpos,applied_velocity_limit, label="steering rate limit")
plt.ylim([-1,max_velocity+1])
plt.xlabel("travel distance",fontsize=15)
plt.ylabel("longitudinal velocity",fontsize=15)
plt.legend()

In [ ]:
plt.plot(np.arange(total_path.shape[0]) * dpos,velocity_limit, label="final velocity limit")
plt.ylim([-1,max_velocity+1])
plt.xlabel("position x",fontsize=15)
plt.ylabel("longitudinal velocity",fontsize=15)
plt.legend()

# velocity smoothing optimization (simplified version)
$$
    \begin{matrix}
    \min & -vel^2 \\
    \mbox{subject to } & 0 \le vel \le vel_{limit} \\ 
    & acc_{min}\le acc \le acc_{max} \\
    & jerk_{min}\le jerk \le jerk_{max}
    \end{matrix}
$$

In [ ]:
n =len(velocity_limit)
v = casadi.SX.sym('v', n)
w = [v]
lbw = [0]*n
ubw = velocity_limit.tolist()


g = []
lbg = []
ubg = []
J = - (v.T @ v)
for i in range(n-1):
    a = (v[i+1]-v[i]) * v[i]/dpos
    g.append(a)
    lbg.append(min_acc)
    ubg.append(max_acc)
for i in range(n-2):
    a1 = (v[i+1]-v[i]) * v[i]/dpos
    a2 = (v[i+2]-v[i+1]) * v[i+1]/dpos
    jerk = (a2-a1) * v[i]/dpos
    g.append(jerk)
    lbg.append(min_jerk)
    ubg.append(max_jerk)

print("check 1")
nlp = {'f': J, 'x': casadi.vertcat(*w), 'g': casadi.vertcat(*g)} 
print("check 2")
solver = casadi.nlpsol('S','ipopt',nlp)
print("check 3")
sol = solver(x0=([0]*n), lbx=lbw, ubx=ubw, lbg=lbg, ubg=ubg)

In [ ]:
v_path = sol["x"].toarray().flatten()
dt_path = dpos/(v_path + 1e-3)
a_path = (v_path[1:]-v_path[:-1]) / (dt_path[:-1])
a_path = np.hstack([a_path,a_path[-1]])
jerk_path = (a_path[1:]-a_path[:-1]) / (dt_path[:-1])
jerk_path = np.hstack([jerk_path,jerk_path[-1]])
dsteer_path = (total_path[1:,-1]-total_path[:-1,-1])/(dt_path[:-1])
dsteer_path = np.hstack([dsteer_path,dsteer_path[-1]])
timestamp = []
t = 0
for i in range(len(dt_path)):
    timestamp.append(1.0*t)
    t += dt_path[i]
timestamp = np.array(timestamp)

trajectory_data=np.zeros((timestamp.shape[0],9))
trajectory_data[:,0] = timestamp
trajectory_data[:,[1,2,4,6]] = total_path
trajectory_data[:,3] = v_path
trajectory_data[:,5] = a_path
trajectory_data[:,7] = jerk_path
trajectory_data[:,8] = dsteer_path

In [ ]:
np.savetxt("mpc_figure_eight_course_data.csv",trajectory_data,delimiter=',')